# CNN with 20 Classes trained on Open Image Validation Set

In [1]:
import numpy as np
import pandas as pd 
import os
#print(os.listdir("../input"))
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import gc

In [2]:
df = pd.read_csv('validation-annotations-human-imagelabels.csv', usecols=[0,2,3])
df = df[df.Confidence == 1]
print(len(df))

classes = np.array(['/m/01g317', '/m/09j2d', '/m/04yx4', '/m/0dzct', '/m/07j7r', '/m/05s2s', '/m/03bt1vf', '/m/07yv9', '/m/0cgh4', '/m/01prls', '/m/09j5n', '/m/0jbk', '/m/0k4j', '/m/05r655', '/m/02wbm', '/m/0c9ph5', '/m/083wq', '/m/0c_jw', '/m/03jm5', '/m/0d4v4'])
li = []
for i in classes:
    li.append(df[df.LabelName == i])
df = pd.concat(li).sample(frac=1).reset_index(drop=True)
del li
gc.collect()
df.head()
#print(len(df))
#tot=0
#for i in range(len(li)):
#    print(len(li[i]))
#    tot=tot+len(li[i])
#print('The total length is ',tot)

365772


,ImageID,LabelName,Confidence
0,e7a151c7f73792d9,/m/0k4j,1
1,d2ba290a5f5201a0,/m/0cgh4,1
2,f4407d452f8fcc4e,/m/05r655,1
3,9bc07c4ed1653e2d,/m/04yx4,1
4,2dd9378cf8e0c8ad,/m/07j7r,1


In [3]:
labels = df.LabelName.tolist()
Imageid = df.ImageID.values
print(type(labels))

<class 'list'>


In [4]:
from keras.preprocessing.image import load_img, img_to_array
from keras.layers import Dense, Conv2D, PReLU, BatchNormalization, MaxPooling2D, Dropout, Flatten
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras.models import load_model, Sequential
from tqdm import tqdm_notebook
from tqdm import tqdm
from keras.optimizers import Adam

/home/akshay/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
df.head(),gc.collect()

(            ImageID  LabelName  Confidence
 0  e7a151c7f73792d9    /m/0k4j           1
 1  d2ba290a5f5201a0   /m/0cgh4           1
 2  f4407d452f8fcc4e  /m/05r655           1
 3  9bc07c4ed1653e2d   /m/04yx4           1
 4  2dd9378cf8e0c8ad   /m/07j7r           1, 0)

In [6]:
X_train = [np.array(load_img('validation/{}.jpg'.format(i),target_size=(100,100), grayscale=True))/255 for i in tqdm(Imageid[10000:20000])]

  0%|          | 0/10000 [00:00<?, ?it/s]/home/akshay/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 10000/10000 [05:49<00:00, 28.60it/s] 


In [7]:
X_Val = [np.array(load_img('validation/{}.jpg'.format(i),target_size=(100,100), grayscale=True))/255 for i in tqdm(Imageid[:2000])]

  0%|          | 0/2000 [00:00<?, ?it/s]/home/akshay/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 2000/2000 [01:04<00:00, 31.01it/s]


In [8]:
#Y_train = labels[10000:20000]
#Y_Val = labels[:2000]
classes = classes.tolist()
Y_train, Y_Val = [], []
for i in tqdm(labels[10000:20000]):
    temp = np.zeros(20)
    classes.index(i)
    temp[classes.index(i)] = 1
    Y_train.append(temp)
    del temp
for i in tqdm(labels[:2000]):
    temp = np.zeros(20)
    temp[classes.index(i)] = 1
    Y_Val.append(temp)
    del temp
Y_train = np.array(Y_train)
Y_Val = np.array(Y_Val)
gc.collect(), Y_train[0]

100%|██████████| 2000/2000 [00:00<00:00, 527718.17it/s]


(0, array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0.]))

In [9]:
nn = Sequential()
nn.add(BatchNormalization(input_shape=(100, 100, 1)))
nn.add(Conv2D(4, kernel_size=(2,2), strides=(1,1)))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Conv2D(8, kernel_size=(2,2), strides=(1,1)))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Conv2D(16, kernel_size=(2,2), strides=(2,2)))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Conv2D(32, kernel_size=(2,2), strides=(1,1)))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Conv2D(32, kernel_size=(2,2), strides=(2,2)))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Conv2D(32, kernel_size=(2,2), strides=(2,2)))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Flatten())
nn.add(Dense(2048))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Dense(1024))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Dense(512))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Dense(128))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Dense(50))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Dense(25))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(0.25))
nn.add(Dense(20, activation='softmax'))

In [10]:
nn.compile(loss=keras.losses.categorical_crossentropy, metrics=['accuracy'], optimizer='adam')
nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 100, 100, 1)       4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 99, 99, 4)         20        
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 99, 99, 4)         39204     
_________________________________________________________________
batch_normalization_2 (Batch (None, 99, 99, 4)         16        
_________________________________________________________________
dropout_1 (Dropout)          (None, 99, 99, 4)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 98, 98, 8)         136       
_________________________________________________________________
p_re_lu_2 (PReLU)            (None, 98, 98, 8)         76832     
__________

In [11]:
X_train = np.array(X_train).reshape((10000,100,100,1))
X_Val = np.array(X_Val).reshape((2000,100,100,1))



In [12]:
gc.collect()

0

In [13]:
nn.fit(X_train, Y_train, validation_data=(X_Val,Y_Val), batch_size=100, epochs=50, verbose=2)

Train on 10000 samples, validate on 2000 samples
Epoch 1/50
 - 204s - loss: 3.3365 - acc: 0.0662 - val_loss: 5.0814 - val_acc: 0.0030
Epoch 2/50
 - 198s - loss: 3.0214 - acc: 0.0892 - val_loss: 4.3835 - val_acc: 0.0030
Epoch 3/50
 - 210s - loss: 2.8526 - acc: 0.1201 - val_loss: 4.4524 - val_acc: 0.0670
Epoch 4/50
 - 202s - loss: 2.7638 - acc: 0.1446 - val_loss: 3.7354 - val_acc: 0.0695
Epoch 5/50
 - 202s - loss: 2.6886 - acc: 0.1554 - val_loss: 4.0170 - val_acc: 0.0700
Epoch 6/50
 - 199s - loss: 2.6268 - acc: 0.1652 - val_loss: 3.3760 - val_acc: 0.0835
Epoch 7/50
 - 198s - loss: 2.5720 - acc: 0.1783 - val_loss: 3.7580 - val_acc: 0.0915
Epoch 8/50
 - 199s - loss: 2.5182 - acc: 0.1924 - val_loss: 3.9630 - val_acc: 0.0965
Epoch 9/50
 - 199s - loss: 2.4771 - acc: 0.1976 - val_loss: 3.4371 - val_acc: 0.1035
Epoch 10/50
 - 199s - loss: 2.4382 - acc: 0.2087 - val_loss: 3.7223 - val_acc: 0.1105
Epoch 11/50
 - 199s - loss: 2.4049 - acc: 0.2171 - val_loss: 3.4608 - val_acc: 0.1150
Epoch 12/50
 -

In [14]:
del X_train, Y_train, X_Val, Y_Val, df
gc.collect()

0

In [25]:
df = pd.read_csv('inclusive_images_stage_1_attributions.csv', usecols=[0], header=None)
df.columns=['image_id']
im=df.image_id.tolist()
df.head()
#print(len(df))
list(df)

['image_id']

In [26]:
print(len(im))
X_test = [np.array(load_img('inclusive_images_stage_1_images/{}.jpg'.format(i),target_size=(100,100), grayscale=True))/255 for i in tqdm_notebook(im)]

32226


/home/akshay/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [28]:
print(len(X_test))
X_test = np.array(X_test).reshape((32226,100,100,1))

32226


In [29]:
pre = nn.predict(X_test).argsort(1)[:,:5]
del X_test
gc.collect()

408

In [36]:
p = []
for it in tqdm(pre):
    p.append(' '.join([classes[int(i)] for i in it]))
print(it)
pre.shape

100%|██████████| 32226/32226 [00:00<00:00, 357749.52it/s]

[18 19  4 17  8]


(32226, 5)

In [31]:
df['labels'] = p
df.head()

,image_id,labels
0,2b2b327132556c767a736b3d,/m/083wq /m/0d4v4 /m/0k4j /m/01prls /m/0cgh4
1,2b2b394755692f303963553d,/m/03jm5 /m/0c_jw /m/07j7r /m/0d4v4 /m/0cgh4
2,2b2b42584e6d445937444d3d,/m/0dzct /m/05r655 /m/03bt1vf /m/0c_jw /m/02wbm
3,2b2b44744e57674270616f3d,/m/0dzct /m/05r655 /m/0d4v4 /m/0c_jw /m/02wbm
4,2b2b4b425a592b683059493d,/m/0d4v4 /m/03jm5 /m/0cgh4 /m/083wq /m/0k4j


In [32]:
df.to_csv('sub.csv', index=False)

## If you find this kernel helpful, please upvote it.
## If you have any questions or suggestions please let me know.